## Import the necessary modules

In [1]:
import os
import mysql.connector as connector
import logging

## Functions
Stored procedures and functions wrap code within the body of a function or procedure for repeated use
- Functions: used to return a specific value
- Stored procedures: Processing, manipulating and modifying data
- Variable: Used to pass values between SQL statements, or between a procedure and a SQL statement. They are created inside or outside a Stored procedure or SELECT statement

## Variables syntax
```sql
@variable_name = value
```

**SET** command  
Assigns a value to a variable within a stored procedure
```sql
SET @variable_name = value
```
To create a variable inside a stored procedure, use the ```declare``` command  

Variables inside a SELECT command
```sql
SELECT @variable_name := value;
SELECT @max_order := MAX(Cost) FROM Orders;
SELECT @max_order
SELECT function() INTO variable_name FROM table_name;
SELECT AVG(Cost) INTO @average_cost FROM Orders;
```


## Parameters
Pass arguments, or values, to a function or procedure from the outside
- Types of Parameters: IN, OUT, INOUT

OUT parameter syntax
```sql
CREATE PROCEDURE GetLowestCost (OUT LowestCost DECIMAL(6, 2))
SELECT MIN(Cost) INTO LowestCost FROM Orders;

CALL GetLowestCost(@order_lowest_cost)
SELECT @order_lowest_cost
```

INOUT parameter syntax
```sql
CREATE PROCEDURE SquareAnumber(INOUT aNumber INT)
BEGIN
    SET aNumber = aNumber * aNumber;
END

SET @x_number = 5
CALL SquareAnumber(@x_number)
```

## User-defined function syntax
```sql
CREATE FUNCTION function_name()
RETURNS datatype DETERMINISTIC
RETURN
```

# Triggers 
A Set of actions available in the form of a stored program invoked when an event occurs. Example includes INSERT, UPDATE AND DELETE. It is associated with a table
```sql
CREATE TRIGGER trigger_name
TRIGGER TYPE
ON table_name FOR EACH ROW
BEGIN 
statement_one
statement_two
END
```

```sql
DROP TRIGGER IF EXISTS schema_name.trigger_name
```

- Each trigger name must be unique within the database
- Dropping a table from a database removes the related triggers
- Useful for maintaining audit trails
- Alternative to constraints: Maintain data integrity and run tasks

## Types of Triggers
- Row-level: Trigger invoked for each row. 100 rows ==> 100 row-level triggers ==> Table.  MySQL only supports this kind of trigger.
- Statement-level: Trigger invoked for each statement. 100 rows ==> Single Trigger ==> Table. This is not supported by MySQL

BEFORE  
Trigger invoked before an action. BEFORE INSERT, BEFORE UPDATE, BEFORE DELETE  
AFTER   
Trigger innvoked after an action. AFTER INSERT, AFTER UPDATE, AFTER DELETE


```sql
CREATE TRIGGER OrderQtyCheck
BEFORE INSERT ON Orders FOR EACH ROW
BEGIN
IF NEW.orderQty < 0 THEN
SET NEW.orderQty = 0;
END IF;
END;
```

```sql
CREATE TRIGGER LogNewOrderInsert
AFTER INSERT ON Orders FOR EACH ROW
BEGIN
INSERT INTO Audits VALUES('AFTER', 'A new order was inserted', 'INSERT');
END;
```

```sql
CREATE TRIGGER AfterDeleteOrder
AFTER INSERT ON Orders FOR EACH ROW
BEGIN
INSERT INTO Audits VALUES('AFTER', CONCAT('Order', OLD.OrderID, ' was deleted at ', CURRENT_TIME(), ' on ', CURRENT_DATE(), 'DELETE'));
END;
```


# Working with MySQL Scheduled Events
A scheduled event is a task that takes place at a specific time according to a schedule
- All Events have a unique name
- Each event contains one or more SQL statements
- Events can occur once or multiple times

```sql
CREATE EVENT IF NOT EXISTS event_name
ON SCHEDULE AT CURRENT_TIMESTAMP [+ INTERVAL]
DO 
Event_body
```

```sql
CREATE EVENT IF NOT EXISTS GenerateRevenueReport
ON SCHEDULE AT CURRENT_TIMESTAMP + INTERVAL 12 HOUR
DO 
BEGIN
INSERT INTO ReportData (OrderID, ClientID, ProductID, Quantity, Cost, Date)
SELECT * FROM Orders WHERE Date BETWEEN '2022-08-01' AND '2022-08-31'
END
```

```sql
CREATE EVENT IF NOT EXISTS GenerateRevenueReport
ON SCHEDULE EVERY INTERVAL STARTS timestamp [+ INTERVAL] ENDS timestamp [+ INTERVAL]
DO 
BEGIN
INSERT INTO ReportData (OrderID, ClientID, ProductID, Quantity, Cost, Date)
SELECT * FROM Orders WHERE Date BETWEEN '2022-08-01' AND '2022-08-31'
END
```

```sql
CREATE EVENT IF NOT EXISTS DailyRestock
ON SCHEDULE EVERY 1 DAY
DO 
BEGIN
IF Products.NumberOfItems < 50 THEN 
UPDATE Products SET NumberOfItems = 50;
END IF;
END
```

# Labs
## Create a logger

In [2]:
logger = logging.getLogger("[Advanced MySQL]")
if os.path.exists("log/advanced-mysql.log"):
  os.remove("log/advanced-mysql.log")
logging.basicConfig(filename='log/advanced-mysql.log', encoding='utf-8', level=logging.DEBUG, format='%(asctime)s ==> %(message)s', datefmt='%m/%d/%Y %I:%M:%S')

## Establish a connection

In [3]:
logger.info("Creating a connection between MySQL and Python")
dbconfig={"user":"root", "password":os.environ["MYSQL_PASSWORD"], "port":33061, "host":"localhost"}
connection=connector.connect(**dbconfig)
print("Connection established between MySQL and Python")
logger.info("Connection established between MySQL and Python")

Connection established between MySQL and Python


## Create a cursor objects

In [4]:
print("Creating cursor object from connection")
logger.info("Creating first cursor object from connection")
cursor = connection.cursor()
print("Cursor object created to communicate with MySQL using Python.")
logger.info("Cursor object created to communicate with MySQL using Python.")

Creating cursor object from connection
Cursor object created to communicate with MySQL using Python.


## Create Database 

In [5]:
# Get a cursor object from the cursor pool
database_name: str = "db_meta_advanced_mysql"
drop_database_query: str = f"""DROP DATABASE IF EXISTS {database_name}"""
cursor.execute(drop_database_query)
logger.info("Dropping Database if it already exists.")

create_database_query: str = f"""CREATE DATABASE IF NOT EXISTS {database_name}"""
print("Creating Database.")
logger.info("Creating Database.")
cursor.execute(create_database_query)
logger.info("Database created.")
print("Database created.")

Creating Database.
Database created.


In [6]:
# Check to see that the database was created
list_of_databases: list = []
cursor.execute("SHOW DATABASES;")
databases = cursor.fetchall()
for database in databases:
    db_name: str = database[0]
    list_of_databases.append(db_name)
    if db_name == database_name:
        print(f"Database '{database_name}' was successfully created")
        logger.info(f"Database '{database_name}' was successfully created.")
        break

# Set the new created database as the database to use
cursor.execute(f"USE {database_name}")
print(f"Database '{database_name}' set for use.")
logger.info(f"Database '{database_name}' set for use.")

Database 'db_meta_advanced_mysql' was successfully created
Database 'db_meta_advanced_mysql' set for use.


## Assert Database contains no Tables

In [7]:
## Expect an empty list to be returned 
show_tables_query = """SHOW TABLES;"""
cursor.execute(show_tables_query)
results = cursor.fetchall()
assert len(results) == 0

## Create Tables

In [8]:
create_orders_query = """CREATE TABLE IF NOT EXISTS tbl_orders (OrderID INT NOT NULL PRIMARY KEY, ClientID VARCHAR(10), ProductID VARCHAR(10), Quantity INT, Cost DECIMAL(6,2), Date DATE);"""
cursor.execute(create_orders_query)
logger.info("tbl_orders table created.")

create_products_query = """CREATE TABLE IF NOT EXISTS tbl_products (ProductID VARCHAR(10), ProductName VARCHAR(100),BuyPrice DECIMAL(6,2), SellPrice DECIMAL(6,2), NumberOfItems INT);"""
cursor.execute(create_products_query)
logger.info("tbl_products table created.")

create_notifications_query = """CREATE TABLE IF NOT EXISTS tbl_notifications (NotificationID INT AUTO_INCREMENT, Notification VARCHAR(255), UpdatedBy VARCHAR(45) NOT NULL,
DateTime TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP, PRIMARY KEY(NotificationID));"""
cursor.execute(create_notifications_query)
logger.info("tbl_notifications table created.")
#ON UPDATE CURRENT_TIMESTAMP,

In [9]:
insert_into_orders_query = """INSERT INTO tbl_orders(OrderID, ClientID, ProductID , Quantity, Cost, Date) VALUES
(1, "Cl1", "P1", 10, 500, "2020-09-01"),  
(2, "Cl2", "P2", 5, 100, "2020-09-05"),  
(3, "Cl3", "P3", 20, 800, "2020-09-03"),  
(4, "Cl4", "P4", 15, 150, "2020-09-07"),  
(5, "Cl3", "P3", 10, 450, "2020-09-08"),  
(6, "Cl2", "P2", 5, 800, "2020-09-09"),  
(7, "Cl1", "P4", 22, 1200, "2020-09-10"),  
(8, "Cl3", "P1", 15, 150, "2020-09-10"),  
(9, "Cl1", "P1", 10, 500, "2020-09-12"),  
(10, "Cl2", "P2", 5, 100, "2020-09-13"),  
(11, "Cl4", "P5", 5, 100, "2020-09-15"), 
(12, "Cl1", "P1", 10, 500, "2022-09-01"),  
(13, "Cl2", "P2", 5, 100, "2022-09-05"),  
(14, "Cl3", "P3", 20, 800, "2022-09-03"),  
(15, "Cl4", "P4", 15, 150, "2022-09-07"),  
(16, "Cl3", "P3", 10, 450, "2022-09-08"),  
(17, "Cl2", "P2", 5, 800, "2022-09-09"),  
(18, "Cl1", "P4", 22, 1200, "2022-09-10"),  
(19, "Cl3", "P1", 15, 150, "2022-09-10"),  
(20, "Cl1", "P1", 10, 500, "2022-09-12"),  
(21, "Cl2", "P2", 5, 100, "2022-09-13"),   
(22, "Cl2", "P1", 10, 500, "2021-09-01"),  
(23, "Cl2", "P2", 5, 100, "2021-09-05"),  
(24, "Cl3", "P3", 20, 800, "2021-09-03"),  
(25, "Cl4", "P4", 15, 150, "2021-09-07"),  
(26, "Cl1", "P3", 10, 450, "2021-09-08"),  
(27, "Cl2", "P1", 20, 1000, "2022-09-01"),  
(28, "Cl2", "P2", 10, 200, "2022-09-05"),  
(29, "Cl3", "P3", 20, 800, "2021-09-03"),  
(30, "Cl1", "P1", 10, 500, "2022-09-01");
"""

cursor.execute(insert_into_orders_query)
connection.commit()


insert_into_products_query = """INSERT INTO tbl_products (ProductID, ProductName, BuyPrice, SellPrice, NumberOfItems)
VALUES ("P1", "Artificial grass bags ", 40, 50, 100),  
("P2", "Wood panels", 15, 20, 250),  
("P3", "Patio slates",35, 40, 60),  
("P4", "Sycamore trees ", 7, 10, 50),  
("P5", "Trees and Shrubs", 35, 50, 75),  
("P6", "Water fountain", 65, 80, 15);"""

cursor.execute(insert_into_products_query)
connection.commit()

## Function to display results

In [10]:
def select_all_query(table_name: str):
    query = f"""SELECT * FROM {table_name};"""
    return query


def display_results(table_column_names: list, results: list):
    table_columns_length = [len(x) for x in table_column_names]
    for result in results:
        for value in range(len(result)):
            row_data = result[value]
            if row_data:
                row_data = str(row_data)
                if len(row_data) > table_columns_length[value]:
                    table_columns_length[value] = len(row_data)
    dashes_plus = ""
    for num in range(len(table_columns_length)):
        dashes_plus = dashes_plus + "+" + '-'*(table_columns_length[num]+2)
    dashes_plus = dashes_plus + "+"
    
    print(dashes_plus)
    
    table_headers = ""
    for num in range(len(table_column_names)):
        table_headers = table_headers + f"| {table_column_names[num]:^{table_columns_length[num]}} "
    table_headers = table_headers + "|"
    print(table_headers)
    
    print(dashes_plus)
    
    for result in results:
        table_row = ""
        for value in range(len(result)):
            row_data = result[value]
            if not row_data:
                if "Field" in table_column_names or "select_type" in table_column_names:
                    row_data = " NULL"
                else:
                    row_data = "None"
            table_row = table_row + "|" + f"{str(row_data):^{table_columns_length[value]+2}}"
        print(table_row + "|")
    print(dashes_plus)
    print(f"{len(results)} rows returned")

def execute_display_query_results(query: str = "", table_column_names: list = [], results: list = []): 
    if query:
        logger.info(f"Executing the query: {query}")
    if len(query) > 2 and (table_column_names or results):
        print("You can only pass in the query alone or the table_column_names and results list")
        assert False
    if query and not table_column_names and not results:
        cursor.execute(query)
        results = cursor.fetchall()    
        table_column_names = cursor.column_names
    
    display_results(table_column_names, results)


In [11]:
execute_display_query_results(select_all_query("tbl_orders"))

print("\n\n")
execute_display_query_results(select_all_query("tbl_products"))

+---------+----------+-----------+----------+---------+------------+
| OrderID | ClientID | ProductID | Quantity |  Cost   |    Date    |
+---------+----------+-----------+----------+---------+------------+
|    1    |   Cl1    |    P1     |    10    | 500.00  | 2020-09-01 |
|    2    |   Cl2    |    P2     |    5     | 100.00  | 2020-09-05 |
|    3    |   Cl3    |    P3     |    20    | 800.00  | 2020-09-03 |
|    4    |   Cl4    |    P4     |    15    | 150.00  | 2020-09-07 |
|    5    |   Cl3    |    P3     |    10    | 450.00  | 2020-09-08 |
|    6    |   Cl2    |    P2     |    5     | 800.00  | 2020-09-09 |
|    7    |   Cl1    |    P4     |    22    | 1200.00 | 2020-09-10 |
|    8    |   Cl3    |    P1     |    15    | 150.00  | 2020-09-10 |
|    9    |   Cl1    |    P1     |    10    | 500.00  | 2020-09-12 |
|   10    |   Cl2    |    P2     |    5     | 100.00  | 2020-09-13 |
|   11    |   Cl4    |    P5     |    5     | 100.00  | 2020-09-15 |
|   12    |   Cl1    |    P1     |

## Task 1
Create a SQL function that prints the cost value of a specific order based on the user input of the OrderID.

In [12]:
create_function_query = """CREATE FUNCTION FindCost(order_id INT) RETURNS DECIMAL(5,2) DETERMINISTIC
RETURN (SELECT Cost FROM tbl_orders WHERE OrderID = order_id);"""

cursor.execute(create_function_query)

In [13]:
select_query = "SELECT FindCost(5);"

execute_display_query_results(select_query)

+-------------+
| FindCost(5) |
+-------------+
|   450.00    |
+-------------+
1 rows returned


## Task 2

Create a stored procedure called GetDiscount(). This stored procedure must return the final cost of the customer’s order after the discount value has been deducted. The discount value is based on the order’s quantity. The stored procedure must have the following specifications:

The procedure should take one parameter that accepts a user input value of an OrderID. 

The procedure must find the order quantity of the specificOrderID. 

If the value of the order quantity is more than or equal to 20 then the procedure should return the new cost after a 20% discount. 

If the value of the order quantity is less than 20 and more than or equal to 10 then the procedure should return the new cost after a 10% discount.

In [14]:
create_procedure_query = """CREATE Procedure GetDiscount(OrderIDInput INT) 
BEGIN 
DECLARE cost_after_discount DECIMAL(7,2); 
DECLARE current_cost DECIMAL(7,2); 
DECLARE order_quantity INT; 
SELECT Quantity INTO order_quantity FROM tbl_orders WHERE OrderID = OrderIDInput; 
SELECT Cost INTO current_cost FROM tbl_orders WHERE OrderID = OrderIDInput; 
IF order_quantity >= 20 THEN
SET cost_after_discount = current_cost - (current_cost * 0.2);              
ELSEIF order_quantity >= 10 THEN
SET cost_after_discount = current_cost - (current_cost * 0.1); 
ELSE SET cost_after_discount = current_cost;
END IF;
SELECT cost_after_discount; 
END
"""

cursor.execute(create_procedure_query)

In [15]:
cursor.callproc("GetDiscount", (5, ))
logger.info("Procedure GetDiscount called with value 5")
results=next(cursor.stored_results())
table_column_names = results.column_names
dataset = results.fetchall()

execute_display_query_results(table_column_names=table_column_names, results=dataset)

+---------------------+
| cost_after_discount |
+---------------------+
|       405.00        |
+---------------------+
1 rows returned


## Task 3
Create an INSERT trigger called ProductSellPriceInsertCheck. This trigger must check if the SellPrice of the product is less than the BuyPrice after a new product is inserted in the Products table. 
If this occurs, then a notification must be added to the Notifications table to inform the sales department. The sales department can then ensure that the incorrect values were not inserted by mistake.
The notification message should be in the following format: A SellPrice less than the BuyPrice was inserted for ProductID + ProductID

In [16]:
create_trigger_query = """CREATE TRIGGER ProductSellPriceInsertCheck
AFTER INSERT ON tbl_products FOR EACH ROW
BEGIN
INSERT INTO tbl_notifications(Notification, UpdatedBy) VALUES(CONCAT('A SellPrice same or less than the BuyPrice was inserted for ProductID ', NEW.ProductID), USER());
END"""
cursor.execute(create_trigger_query)

In [17]:
insert_into_products_query = """INSERT INTO tbl_products (ProductID, ProductName, BuyPrice, SellPrice, NumberOfItems) VALUES('P7', 'Product P7', 40, 40, 100);"""
cursor.execute(insert_into_products_query)
connection.commit()

execute_display_query_results(select_all_query("tbl_products"))
print("\n")
execute_display_query_results(select_all_query("tbl_notifications"))

+-----------+------------------------+----------+-----------+---------------+
| ProductID |      ProductName       | BuyPrice | SellPrice | NumberOfItems |
+-----------+------------------------+----------+-----------+---------------+
|    P1     | Artificial grass bags  |  40.00   |   50.00   |      100      |
|    P2     |      Wood panels       |  15.00   |   20.00   |      250      |
|    P3     |      Patio slates      |  35.00   |   40.00   |      60       |
|    P4     |    Sycamore trees      |   7.00   |   10.00   |      50       |
|    P5     |    Trees and Shrubs    |  35.00   |   50.00   |      75       |
|    P6     |     Water fountain     |  65.00   |   80.00   |      15       |
|    P7     |       Product P7       |  40.00   |   40.00   |      100      |
+-----------+------------------------+----------+-----------+---------------+
7 rows returned


+----------------+--------------------------------------------------------------------------+----------------+--------------

## Task 4
Create an UPDATE trigger called ProductSellPriceUpdateCheck. This trigger must check that products are not updated with a SellPrice that is less than or equal to the BuyPrice. 
If this occurs, add a notification to the Notifications table for the sales department so they can ensure that product prices were not updated with the incorrect values. 
This trigger sends a notification to the Notifications table that warns the sales department of the issue.

The notification message should be in the following format: ProductID + was updated with a SellPrice of  + SellPrice + which is the same or less than the BuyPrice

In [18]:
create_trigger_query = """CREATE TRIGGER ProductSellPriceUpdateCheck
AFTER UPDATE ON tbl_products FOR EACH ROW
BEGIN
IF NEW.SellPrice < OLD.BuyPrice THEN
INSERT INTO tbl_notifications(Notification, UpdatedBy) VALUES(CONCAT(OLD.ProductID, ' was updated with a SellPrice of ', NEW.SellPrice, ' which is the same or less than the BuyPrice'), USER());
END IF;
END;"""
cursor.execute(create_trigger_query)

In [19]:
update_products_query = """UPDATE tbl_products SET SellPrice = 60 WHERE ProductID = 'P6';"""
cursor.execute(update_products_query)
connection.commit()

execute_display_query_results(select_all_query("tbl_products"))
print("\n")
execute_display_query_results(select_all_query("tbl_notifications"))

+-----------+------------------------+----------+-----------+---------------+
| ProductID |      ProductName       | BuyPrice | SellPrice | NumberOfItems |
+-----------+------------------------+----------+-----------+---------------+
|    P1     | Artificial grass bags  |  40.00   |   50.00   |      100      |
|    P2     |      Wood panels       |  15.00   |   20.00   |      250      |
|    P3     |      Patio slates      |  35.00   |   40.00   |      60       |
|    P4     |    Sycamore trees      |   7.00   |   10.00   |      50       |
|    P5     |    Trees and Shrubs    |  35.00   |   50.00   |      75       |
|    P6     |     Water fountain     |  65.00   |   60.00   |      15       |
|    P7     |       Product P7       |  40.00   |   40.00   |      100      |
+-----------+------------------------+----------+-----------+---------------+
7 rows returned


+----------------+--------------------------------------------------------------------------------------+----------------+--

## Task 5:

Create a DELETE trigger called NotifyProductDelete. This trigger must insert a notification in the Notifications table for the sales department after a product has been deleted from the Products table.

The notification message should be in the following format: The product with a ProductID  + ProductID + was deleted

In [20]:
create_trigger_query = """CREATE TRIGGER NotifyProductDelete
AFTER DELETE ON tbl_products FOR EACH ROW
BEGIN
INSERT INTO tbl_notifications(Notification, UpdatedBy) VALUES(CONCAT('The product with the ProductID: ', OLD.ProductID, ' was deleted'), USER());
END;"""
cursor.execute(create_trigger_query)

In [21]:
delete_from_products_query = """DELETE FROM tbl_products WHERE ProductID = 'P7';"""
cursor.execute(delete_from_products_query)
connection.commit()

execute_display_query_results(select_all_query("tbl_products"))
print("\n")
execute_display_query_results(select_all_query("tbl_notifications"))

+-----------+------------------------+----------+-----------+---------------+
| ProductID |      ProductName       | BuyPrice | SellPrice | NumberOfItems |
+-----------+------------------------+----------+-----------+---------------+
|    P1     | Artificial grass bags  |  40.00   |   50.00   |      100      |
|    P2     |      Wood panels       |  15.00   |   20.00   |      250      |
|    P3     |      Patio slates      |  35.00   |   40.00   |      60       |
|    P4     |    Sycamore trees      |   7.00   |   10.00   |      50       |
|    P5     |    Trees and Shrubs    |  35.00   |   50.00   |      75       |
|    P6     |     Water fountain     |  65.00   |   60.00   |      15       |
+-----------+------------------------+----------+-----------+---------------+
6 rows returned


+----------------+--------------------------------------------------------------------------------------+----------------+---------------------+
| NotificationID |                                     No

In [22]:
insert_into_products_query = """INSERT INTO tbl_products (ProductID, ProductName, BuyPrice, SellPrice, NumberOfItems) 
VALUES ('P8', 'Product P8', 40, 40, 100),
('P9', 'Product P9', 40, 40, 100);"""
cursor.execute(insert_into_products_query)
connection.commit()

execute_display_query_results(select_all_query("tbl_products"))
print("\n")
execute_display_query_results(select_all_query("tbl_notifications"))

+-----------+------------------------+----------+-----------+---------------+
| ProductID |      ProductName       | BuyPrice | SellPrice | NumberOfItems |
+-----------+------------------------+----------+-----------+---------------+
|    P1     | Artificial grass bags  |  40.00   |   50.00   |      100      |
|    P2     |      Wood panels       |  15.00   |   20.00   |      250      |
|    P3     |      Patio slates      |  35.00   |   40.00   |      60       |
|    P4     |    Sycamore trees      |   7.00   |   10.00   |      50       |
|    P5     |    Trees and Shrubs    |  35.00   |   50.00   |      75       |
|    P6     |     Water fountain     |  65.00   |   60.00   |      15       |
|    P8     |       Product P8       |  40.00   |   40.00   |      100      |
|    P9     |       Product P9       |  40.00   |   40.00   |      100      |
+-----------+------------------------+----------+-----------+---------------+
8 rows returned


+----------------+----------------------------

In [23]:
update_products_query = """UPDATE tbl_products SET SellPrice = 25 WHERE ProductID = 'P3';"""
cursor.execute(update_products_query)
connection.commit()


execute_display_query_results(select_all_query("tbl_products"))
print("\n")
execute_display_query_results(select_all_query("tbl_notifications"))

+-----------+------------------------+----------+-----------+---------------+
| ProductID |      ProductName       | BuyPrice | SellPrice | NumberOfItems |
+-----------+------------------------+----------+-----------+---------------+
|    P1     | Artificial grass bags  |  40.00   |   50.00   |      100      |
|    P2     |      Wood panels       |  15.00   |   20.00   |      250      |
|    P3     |      Patio slates      |  35.00   |   25.00   |      60       |
|    P4     |    Sycamore trees      |   7.00   |   10.00   |      50       |
|    P5     |    Trees and Shrubs    |  35.00   |   50.00   |      75       |
|    P6     |     Water fountain     |  65.00   |   60.00   |      15       |
|    P8     |       Product P8       |  40.00   |   40.00   |      100      |
|    P9     |       Product P9       |  40.00   |   40.00   |      100      |
+-----------+------------------------+----------+-----------+---------------+
8 rows returned


+----------------+----------------------------

# Optimization  
Database optimization is improving database performance to reduce the execution time of a query  
- Target columns with SELECT
- Use of functions and wildcards
- Use of INNER JOINS with tables
- Deploy DISTINCT and UNION clauses
- Use the ORDER BY clause to sort results

Optimizing database SELECT statements
- Target required columns
- Avoid functions: Avoid using functions in predicates(A predicate is an expression that returns a true or false value. An example of this is WHERE clause conditions)
- Avoid leading wildcards: LIKE %____
- Use INNER JOIN where possible
- Make use of distinct and union clauses only when necessary: USE UNION ALL instead of UNION

In [24]:
create_table_customers_query = """CREATE TABLE IF NOT EXISTS tbl_customers(CustomerID INT NOT NULL PRIMARY KEY, FullName VARCHAR(100) NOT NULL, PhoneNumber INT NOT NULL UNIQUE);"""
cursor.execute(create_table_customers_query)
print("Table tbl_customers successfully created")
logger.info("Table tbl_customers successfully created")

## Populate the tables with Data
insert_into_tbl_customers = """INSERT INTO tbl_customers(CustomerID, FullName, PhoneNumber) VALUES 
(1, "Vanessa McCarthy", 0757536378), (2, "Marcos Romero", 0757536379), (3, "Hiroki Yamane", 0757536376), (4, "Anna Iversen", 0757536375), (5, "Diana Pinto", 0757536374),     
(6, "Altay Ayhan", 0757636378), (7, "Jane Murphy", 0753536379), (8, "Laurina Delgado", 0754536376), (9, "Mike Edwards", 0757236375), (10, "Karl Pederson", 0757936374);"""
logger.info("Inserting records into the tbl_customers table")
cursor.execute(insert_into_tbl_customers)
connection.commit()

create_table_customers_query = """CREATE TABLE IF NOT EXISTS customers(CustomerID INT NOT NULL PRIMARY KEY, FullName VARCHAR(100) NOT NULL UNIQUE, PhoneNumber INT NOT NULL UNIQUE);"""
cursor.execute(create_table_customers_query)
print("Table customers successfully created")
logger.info("Table customers successfully created")

## Populate the tables with Data
insert_into_tbl_customers = """INSERT INTO customers(CustomerID, FullName, PhoneNumber) VALUES 
(1, "Vanessa McCarthy", 0757536378), (2, "Marcos Romero", 0757536379), (3, "Hiroki Yamane", 0757536376), (4, "Anna Iversen", 0757536375), (5, "Diana Pinto", 0757536374),     
(6, "Altay Ayhan", 0757636378), (7, "Jane Murphy", 0753536379), (8, "Laurina Delgado", 0754536376), (9, "Mike Edwards", 0757236375), (10, "Karl Pederson", 0757936374);"""
logger.info("Inserting records into the tbl_customers table")
cursor.execute(insert_into_tbl_customers)
connection.commit()

Table tbl_customers successfully created
Table customers successfully created


In [25]:
explain_query = """EXPLAIN SELECT * FROM tbl_customers;"""
execute_display_query_results(explain_query)

+----+-------------+---------------+------------+------+---------------+-----+---------+-----+------+----------+-------+
| id | select_type |     table     | partitions | type | possible_keys | key | key_len | ref | rows | filtered | Extra |
+----+-------------+---------------+------------+------+---------------+-----+---------+-----+------+----------+-------+
| 1  |   SIMPLE    | tbl_customers |    NULL    | ALL  |      NULL     | NULL|   NULL  | NULL|  10  |  100.0   |  NULL |
+----+-------------+---------------+------------+------+---------------+-----+---------+-----+------+----------+-------+
1 rows returned


In [26]:
# """CREATE TABLE IF NOT EXISTS tbl_customers(CustomerID INT NOT NULL PRIMARY KEY, FullName VARCHAR(100) NOT NULL, PhoneNumber INT NOT NULL UNIQUE);"""
explain_query = """EXPLAIN SELECT * FROM tbl_customers WHERE FullName = 'Marcos Romero';"""
execute_display_query_results(explain_query)

+----+-------------+---------------+------------+------+---------------+-----+---------+-----+------+----------+-------------+
| id | select_type |     table     | partitions | type | possible_keys | key | key_len | ref | rows | filtered |    Extra    |
+----+-------------+---------------+------------+------+---------------+-----+---------+-----+------+----------+-------------+
| 1  |   SIMPLE    | tbl_customers |    NULL    | ALL  |      NULL     | NULL|   NULL  | NULL|  10  |   10.0   | Using where |
+----+-------------+---------------+------------+------+---------------+-----+---------+-----+------+----------+-------------+
1 rows returned


In [27]:
# """CREATE TABLE IF NOT EXISTS customers(CustomerID INT NOT NULL PRIMARY KEY, FullName VARCHAR(100) NOT NULL UNIQUE, PhoneNumber INT NOT NULL UNIQUE);"""
explain_query = """EXPLAIN SELECT * FROM customers WHERE FullName = 'Marcos Romero';"""
execute_display_query_results(explain_query)

+----+-------------+-----------+------------+-------+---------------+----------+---------+-------+------+----------+-------+
| id | select_type |   table   | partitions | type  | possible_keys |   key    | key_len |  ref  | rows | filtered | Extra |
+----+-------------+-----------+------------+-------+---------------+----------+---------+-------+------+----------+-------+
| 1  |   SIMPLE    | customers |    NULL    | const |   FullName    | FullName |   402   | const |  1   |  100.0   |  NULL |
+----+-------------+-----------+------------+-------+---------------+----------+---------+-------+------+----------+-------+
1 rows returned


__Column 2: select_type__.  
This column displays the type of SELECT query to be executed.
- __SIMPLE__: Simple SELECT query without any subqueries or UNIONs     
- __PRIMARY__: The SELECT is in the outermost query in a JOIN    
- __DERIVED__: The SELECT is part of a subquery within a FROM clause    
- __SUBQUERY__: The first SELECT in a subquery    
- __DEPENDENT SUBQUERY__: The SELECT statements is a subquery dependent on an outer query    
- __UNCACHEABLE SUBQUERY__: Subquery which is not cacheable (there are certain conditions for a query to be cacheable)    
- __UNION__: The SELECT query is the second or later statement of a UNION    
- __DEPENDENT UNION__: The second or later SELECT of a UNION is dependent on an outer query    
- __UNION RESULT__: The SELECT is a result of a UNION

__Column 04: Partitions__.  
This column displays the partition in which the data resides (the area of physical storage that's scanned). Partitioning allows for the distribution of portions of table data across the file system. If the queries access only a fraction of table data, then there's less records to scan and queries can execute faster. However, partitioning is more meaningful when dealing with large data sets.

__Column 05: type__.  
Scanning the table means performing a search operation or finding matches specified by the SELECT query. The following table outlines a list of the most prominent possible values:
- __system__: The table has only one row or zero rows. This return value typically indicates that the search was performed on a system table.    
- __const__: Indicates that the value of the searched column can be treated as a constant (there is one row matching the query)    
- __eq_ref__: Indicates that the index is clustered and is being used by the operation (either the index is a PRIMARY KEY or UNIQUE INDEX with all key columns defined as NOT NULL)    
- __ref__: Indicates that the indexed column was accessed using an equality operator.  
- __full text__: The scan uses the table’s FULLTEXT index. Full-text indexes are created on text-based columns (CHAR , VARCHAR , or TEXT columns)    
- __index__: The entire index is scanned to find a match for the query    
- __all__: The entire table is scanned to find matching rows. This is the worst scan type and usually indicates the lack of appropriate indexes on the table.
  
The query returns a value of ALL. This indicates that MySQL scans the entire table (each row) to find the matching rows.

__Column 06: possible_keys__  
This column shows the keys that can be used by MySQL to find rows from the table. However, these keys may or may not be used in practice. If the column value is NULL, then it indicates that no relevant indexes are found.
The Lucky Shrub example has a NULL value, which indicates that there are no keys or indexes in the table which MySQL can use to find or filter rows. This indicates a problem that needs to be addressed.

__Column 07: key__  
Indicates the actual index used by MySQL. This column returned a NULL value. This means there’s no index in the table that can be used by the optimizer. This is a problem that needs to be addressed, because an index is required for optimization.

__Column 08: key_len__  
This column indicates the length of the index the Query Optimizer chooses to use. For example, a key_len value of 4 means it requires memory to store four characters. 
This database returns a NULL value again because there is no index that can be used by this query.

__Column 09: ref__
This column shows which table columns have been compared to the index to perform the search. A value of const means a constant, while a value of func means that the value that was used was derived from a function.
In this database, no columns or constants are compared against an index, because the table doesn’t have one. Therefore, the value is NULL. This once again highlights the problem of not using an index in a table.

__Column 10: rows__   
Lists the number of records that were examined to produce the output.
The results indicate that there were 10 records examined by the query. This means that every row in the table was examined. This is an inefficient use of the database's resources. 
If there were hundreds, or even thousands of records in a table, then an inefficient query would have to examine each one. This would take much longer, and use more resources, than an efficient query which only needs to examine the required records.

__Column 11: filtered__
This column indicates an approximate percentage of the number of table rows that have been filtered by a specified condition. The higher the percentage is, the better the performance of the query.
In the query, 10% of the rows in the Clients table are filtered by the WHERE clause condition.

__Column 12: Extra__
Contains additional information regarding the query execution plan. Pay attention if there are values like  Using temporary or Using filesort in this column as they indicate a problematic query. Let's take a quick look at what these values mean.
Using temporary: This value Indicates that MySQL needs to create a temporary table to hold the result of this query. This typically happens if the query contains GROUP BY and ORDER BY clauses that list columns differently (In other words, when the columns listed in the GROUP BY and ORDER BY clauses are different).

Using filesort: This value indicates that MySQL must perform an extra pass to determine how to retrieve the rows in sorted order. The sort is performed by going through all rows according to the type (mentioned earlier) and storing the sort key and pointer to the row for all rows that match the WHERE clause. The keys then are sorted, and the rows are retrieved in sorted order. 

The example has a value of Using where. This indicates that MySQL first reads each row within the table before it filters them. This is not an efficient way for a query to make use of the WHERE clause. If an index was defined on the Clients table, then the value here would be different.

Planning a more efficient SELECT query
The use of an index on the required column, or columns, results in more efficient SELECT queries that processes records faster using less resources.

In [28]:
create_index_query = """CREATE INDEX IdxFullName ON tbl_customers(FullName);"""
cursor.execute(create_index_query)

explain_query = """EXPLAIN SELECT * FROM tbl_customers WHERE FullName = 'Karl Pederson';"""
execute_display_query_results(explain_query)

+----+-------------+---------------+------------+------+---------------+-------------+---------+-------+------+----------+-------+
| id | select_type |     table     | partitions | type | possible_keys |     key     | key_len |  ref  | rows | filtered | Extra |
+----+-------------+---------------+------------+------+---------------+-------------+---------+-------+------+----------+-------+
| 1  |   SIMPLE    | tbl_customers |    NULL    | ref  |  IdxFullName  | IdxFullName |   402   | const |  1   |  100.0   |  NULL |
+----+-------------+---------------+------------+------+---------------+-------------+---------+-------+------+----------+-------+
1 rows returned


In [29]:
create_index_query = """CREATE INDEX IdxFullName ON customers(FullName);"""
cursor.execute(create_index_query)

explain_query = """EXPLAIN SELECT * FROM customers WHERE FullName = 'Karl Pederson';"""
execute_display_query_results(explain_query)

+----+-------------+-----------+------------+-------+----------------------+----------+---------+-------+------+----------+-------+
| id | select_type |   table   | partitions | type  |    possible_keys     |   key    | key_len |  ref  | rows | filtered | Extra |
+----+-------------+-----------+------------+-------+----------------------+----------+---------+-------+------+----------+-------+
| 1  |   SIMPLE    | customers |    NULL    | const | FullName,IdxFullName | FullName |   402   | const |  1   |  100.0   |  NULL |
+----+-------------+-----------+------------+-------+----------------------+----------+---------+-------+------+----------+-------+
1 rows returned


## Index
__Primary Index__, also called a clustered index: A primary index is an index that is stored within the table itself. It's generated automatically once you create a table that contains a primary or unique key, the index enforces the order of rows in the table within the table itself.  
__Secondary or a non-clustered index__: A secondary index is created using the My SQL create index statement. The syntax begins with create index, then write the name of the index. A commonly used approach is to write the name of the column you want to create the index on, prefaced by idx for index. Next, use the ON keyword to assign the index to a table. Finally, add a pair of parenthesis and write a list of columns that the index is to be used against. An index can be created using one or more columns from a table, but you should only create indexes on columns that you'll frequently perform searches against. This is because when you update or insert data into the table, that same data must also be added to or updated within the index, which takes time